In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, GlobalAveragePooling1D, AveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dropout, BatchNormalization
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf

In [5]:
# resetting the seeds for reproducibility
def reset_random_seeds():
    n = 1
    os.environ['PYTHONHASHSEED'] = str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    random.seed(n)

reset_random_seeds()

# import data
df = pd.read_csv('onchain_data_new.csv')
df1 = pd.read_csv('all_data.csv')

df = df[df['timestamp'] >= '2013-03-11'].reset_index(drop=True)

#onchain_data_new
#TA_data
#all_data
#Boruta_data
#Boruta_TA_data
#Boruta_onchain_data
df.shape

(3758, 93)

In [7]:
# separate the inputs and target
X = df.drop('timestamp', axis=1)

# create binary classification for price movement. this assigns 1 to y if price will move upward next day.
price = pd.DataFrame()
price['today'] = df1['price-ohlc-usd-c']
price['next day'] = price['today'].shift(-1)
y = price['next day']

# Drop the last row where 'next day' would be NaN after shifting
X = X[:-1]
y = y.dropna()

# separate training data from testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
# scale the input data
scaler = StandardScaler()

# Reshape X_train and X_test if they are 1D
if X_train.ndim == 1:
    X_train = X_train.to_numpy().reshape(-1, 1)
if X_test.ndim == 1:
    X_test = X_test.to_numpy().reshape(-1, 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# reshape the input data for CNN-LSTM (samples, timesteps, features)
def create_sequences(data, timesteps):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

timesteps = 5
X_train_reshaped = create_sequences(X_train_scaled, timesteps)
X_test_reshaped = create_sequences(X_test_scaled, timesteps)
y_train = y_train[timesteps - 1:]
y_test = y_test[timesteps - 1:]

In [11]:
X_train_reshaped.shape

(3001, 5, 92)

In [5]:
# define the CNN-LSTM model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, X_train_reshaped.shape[2])))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=1))
model.add(Dropout(0.5))
model.add(LSTM(units=128, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(LSTM(units=80))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_absolute_error', metrics=['mean_absolute_error', 'mean_squared_error'])

In [6]:
# train with the training dataset
early_stopping = EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=10, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
270/270 [==============================] - 10s 8ms/step - loss: 3465.5425 - mean_absolute_error: 3465.5425 - mean_squared_error: 27444656.0000 - val_loss: 31414.5723 - val_mean_absolute_error: 31414.5723 - val_mean_squared_error: 1320167680.0000
Epoch 2/100
270/270 [==============================] - 1s 5ms/step - loss: 3451.7263 - mean_absolute_error: 3451.7263 - mean_squared_error: 27358494.0000 - val_loss: 31397.6680 - val_mean_absolute_error: 31397.6680 - val_mean_squared_error: 1319149184.0000
Epoch 3/100
270/270 [==============================] - 1s 5ms/step - loss: 3413.7686 - mean_absolute_error: 3413.7686 - mean_squared_error: 27083704.0000 - val_loss: 31341.4023 - val_mean_absolute_error: 31341.4023 - val_mean_squared_error: 1315670016.0000
Epoch 4/100
270/270 [==============================] - 1s 5ms/step - loss: 3350.7600 - mean_absolute_error: 3350.7600 - mean_squared_error: 26565058.0000 - val_loss: 31235.8535 - val_mean_absolute_error: 31235.8535 - val_mean_sq

In [7]:
# predict with testing dataset
y_pred = model.predict(X_test_reshaped)

# evaluate the prediction performance
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

24/24 [==============================] - 1s 3ms/step
Root Mean Squared Error: 25021.858585552665
Mean Absolute Error: 21342.940216288018
R^2 Score: -2.8353338567047954


In [8]:
time = df['timestamp']
time_train, time_test = train_test_split(time, test_size=0.2, shuffle=False)
time_test = time_test[timesteps:]

# Flatten y_pred to be a 1-dimensional array
y_pred_flat = y_pred.flatten()

# Create a DataFrame with columns time_test, y_test, and y_pred
pred_res = pd.DataFrame({'date': time_test, 'actual': y_test.values, 'prediction': y_pred_flat, 'value': price['today'][-748:]})

pred_res.to_csv('pred2/cnnlstm_uni_data.csv', index=False)